In [2]:
# import necessary packages
import os, sys
sys.path.append(r'/elo_bench')
from datamodel import QuestionCollection

import pandas as pd
from pathlib import Path
from collections import defaultdict

In [5]:
"""convert gemini answers to the designed format csv"""
if False:
    gemini_answers = pd.read_json(r'./gemini_pro_answer_20573_20231228.jsonl', lines=True)

    # take 'question' as row index, and 'answer' as cell of ['question', 'gemini']
    gemini_answers = gemini_answers.set_index('question')
    gemini_answers = gemini_answers.rename(columns={'answer': 'gemini'})
    gemini_answers.to_csv(r'./q_and_as gemini.csv')
    gemini_answers

,gemini
question,
How does the scoring system in Wordle work?,1. **Correct Letter in Correct Position (Green...
Can you explain the popularity of Wordle and why it's so addictive?,1. **Simplicity and Accessibility:**\n - Wor...
What are some tips for beginners trying to master Wordle?,1. **Start with Common Letters:** Begin by gue...
"Is their a way to cheat on Wordle, or is it all about your search skills?",While Wordle is designed to be a game of skill...
Can you tell me some interesting facts about Queen Elizabeth that most people don't know?,1. **She's a trained mechanic.** During World ...
...,...
"Assume we have a band-limited signal and an ideal sampling function (Dirac Comb) with frequency twice that of the band-limited signal. We know our signal is band-limited because we filtered it with an ideal brick-wall filter with a pass-band equal to the band-width of the signal; thus the signal is strictly band-limited. Using LaTeX to beautify your mathematical expressions, what is the spectrum of the sampled signal from a purely mathematical point of view? I'm not interested in practical or real scenarios.",Let \(x(t)\) be the band-limited signal and \(...
write a grant specific aims page on social determinants of health and endometrial cancer,**Specific Aims**\n\n1. **To examine the assoc...
Please create a meal plan for a family of 4. Two adults and two children. Please ensure we have one meat free day per week. Ensure that meals are quick and easy to prepare.,**Day 1: Meatless Monday**\n\n* Breakfast: Oat...


In [2]:
"""list csv splits models name with answers count and percentage"""
if True:
    split_csv_files = [
        r'./q_and_as 2a100.csv',
        r'./q_and_as 1503.csv',
        r'./q_and_as 1673.csv',
        r'./q_and_as 1691', # √
        r'./q_and_as gemini.csv', # √
        r'./q_and_as 2a100 split.csv'
    ]

    for split_csv in split_csv_files:
        print(f'Printing information in {split_csv}...')
        answer_split = pd.read_csv(split_csv, engine='python', keep_default_na=False, na_values=['NaN', 'NULL'])
        skip_columns = ['Unnamed: 0', 'question']
        models = [x for x in answer_split.columns.tolist() if not x in (skip_columns)]
        for model in models:
            print(f'{len(answer_split[model].tolist())} answers for {model}')

Printing information in ./q_and_as 2a100.csv...
21259 answers for lmsys/vicuna-7b-v1.5
21259 answers for gpt-4-turbo
21259 answers for gpt-35-turbo
21259 answers for lmsys/vicuna-13b-v1.5
21259 answers for lmsys/vicuna-33b-v1.3
21259 answers for meta-llama/Llama-2-7b-chat-hf
21259 answers for meta-llama/Llama-2-13b-chat-hf
21259 answers for huggyllama/llama-13b
21259 answers for meta-llama/Llama-2-70b-chat-hf
21259 answers for tiiuae/falcon-7b-instruct
21259 answers for tiiuae/falcon-40b-instruct
21259 answers for mosaicml/mpt-7b-chat
21259 answers for WizardLM/WizardLM-13B-V1.2
21259 answers for Xwin-LM/Xwin-LM-7B-V0.1
21259 answers for chavinlo/alpaca-13b
21259 answers for WizardLM/WizardLM-7B-V1.0
21259 answers for chavinlo/alpaca-native
21259 answers for Xwin-LM/Xwin-LM-13B-V0.1
21259 answers for mosaicml/mpt-30b-chat
21259 answers for huggyllama/llama-7b
21259 answers for HuggingFaceH4/zephyr-7b-beta
21259 answers for huggyllama/llama-30b
Printing information in ./q_and_as 1503.cs

In [4]:
"""Merge the answers for dataset"""
dataset_dir = r'/elo_bench/data/google_quora_alpaca_sharegpt_chatlm_clean_20772'
questions = QuestionCollection.read_csv(Path(dataset_dir)/'questions.csv').questions
question_set = set(questions)

print(f'{len(questions)} questions.')

split_csv_files = [
    r'./q_and_as 2a100.csv',
    r'./q_and_as 1503.csv',
    r'./q_and_as 1673.csv',
    r'./q_and_as 1691', # √
    r'./q_and_as gemini.csv', # √
    r'./q_and_as 2a100 split.csv'
]


q_and_as_dict = defaultdict(lambda: defaultdict(str)) #  a_and_as_dict['question']['model']=ans
for split_csv in split_csv_files:
    print(f'Printing information in {split_csv}...')
    answer_split = pd.read_csv(split_csv, keep_default_na=False, na_values=['NaN'], engine='python')
    # answer_splits.append(answer_split)
    skip_columns = ['Unnamed: 0', 'question']
    models = [str(x) for x in answer_split.columns.tolist() if not x in (skip_columns)]
    for model in models:
        print(model)
        for index, row in answer_split.iterrows():
            question = row['question']
            model_answer = row[model]
            # print(f'{question}:{model_answer}')
            q_and_as_dict[question][model] = model_answer

reformat_matrix = []
for question_key, model_answers in q_and_as_dict.items():
    if question_key not in question_set:
        continue
    item = {
        'question': question_key,
    }
    for model_name, ans in model_answers.items():
        item[model_name] = ans
    
    reformat_matrix.append(item)
    
    
df = pd.DataFrame.from_dict(reformat_matrix)
df
df.to_csv(r'./merge.csv', na_rep='NaN')
# from functools import reduce   
# df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['question'],
#                                             how='outer'), answer_splits).fillna('void')
# df_merged = pd.concat(answer_splits, join='inner', keys=['question'], axis=1, ig)

# df_merged
    # skip_columns = ['Unnamed: 0', 'question']
    # models = [x for x in answer_split.columns.tolist() if not x in (skip_columns)]
    # for model in models:
    #     for _, row in answer_split.iterrows():
    #         question = row.index
    #         model_answer = row[model]
            
        # answer = answer_split[model].tolist()
        # print(f'{len(answer_split[model].tolist())} answers for {model}')

[DEBUG] Removed 48 repeat questions.


20772 questions.
Printing information in ./q_and_as 2a100.csv...
lmsys/vicuna-7b-v1.5
gpt-4-turbo
gpt-35-turbo
lmsys/vicuna-13b-v1.5
lmsys/vicuna-33b-v1.3
meta-llama/Llama-2-7b-chat-hf
meta-llama/Llama-2-13b-chat-hf
huggyllama/llama-13b
meta-llama/Llama-2-70b-chat-hf
tiiuae/falcon-7b-instruct
tiiuae/falcon-40b-instruct
mosaicml/mpt-7b-chat
WizardLM/WizardLM-13B-V1.2
Xwin-LM/Xwin-LM-7B-V0.1
chavinlo/alpaca-13b
WizardLM/WizardLM-7B-V1.0
chavinlo/alpaca-native
Xwin-LM/Xwin-LM-13B-V0.1
mosaicml/mpt-30b-chat
huggyllama/llama-7b
HuggingFaceH4/zephyr-7b-beta
huggyllama/llama-30b
Printing information in ./q_and_as 1503.csv...
Xwin-LM/Xwin-LM-13B-V0.1
Xwin-LM/Xwin-LM-7B-V0.1
WizardLM/WizardLM-7B-V1.0
WizardLM/WizardLM-13B-V1.2
mosaicml/mpt-7b-chat
mosaicml/mpt-30b-chat
HuggingFaceH4/zephyr-7b-beta
Printing information in ./q_and_as 1673.csv...
huggyllama/llama-7b
huggyllama/llama-13b
huggyllama/llama-30b
chavinlo/alpaca-native
chavinlo/alpaca-13b
Printing information in ./q_and_as 1691...
Huggi

In [1]:
"""list csv splits models name with answers count and percentage"""
if True:
    split_csv_files = [
        r'./merge.csv',
    ]

    for split_csv in split_csv_files:
        print(f'Printing information in {split_csv}...')
        answer_split = pd.read_csv(split_csv, engine='python', keep_default_na=False, na_values=['NaN', 'NULL'])
        skip_columns = ['Unnamed: 0', 'question']
        models = [x for x in answer_split.columns.tolist() if not x in (skip_columns)]
        for model in models:
            model_answers = answer_split[model]
            model_answers = model_answers[~model_answers.isna()]
            print(f'{len(model_answers.tolist())} answers for {model}')

Printing information in ./merge.csv...


NameError: name 'pd' is not defined